<a href="https://colab.research.google.com/github/MatteoZanella/NLU-assignement-1/blob/main/NLU_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLU assignment n.1

## Part A: Working with Dependency Graphs
The objective of the assignment is to learn how to work with dependency graphs by defining functions.

Read [spaCy documentation on dependency parser](https://spacy.io/api/dependencyparser) to learn provided methods.

In [2]:
# Imports
import spacy
from spacy import displacy

# HELPER FUNCTIONS
# get the doc from a sentence
def doc_of(sentence: str) -> spacy.tokens.Doc:
  nlp = spacy.load("en_core_web_sm")
  return nlp(sentence)

In [21]:
# Example sentence (for testing)
example = "Credit and mortgage account holders must submit their requests."
# Example sentence visualization
displacy.render(doc_of(example), style='dep', jupyter=True)

### Task A1
Extract a path of dependency relations from the ROOT to a token

The best way to match is from below

In [ ]:
from collections import deque


def dependency_path(sentence: str):
  doc = doc_of(sentence)
  root = doc[:].root
  paths = {root: []}
  tokens = deque([root])
  # BFS search from the root
  while tokens:
    token = tokens.popleft()
    paths[token] = [*paths[token.head], token.dep_]
    tokens.extend(token.children)
  return paths

#### Testing A1

In [ ]:
# Task A1 testing
paths = dependency_path(example)
for token in paths:
  print(f"{token}: {paths[token]}")

submit: ['ROOT']
holders: ['ROOT', 'nsubj']
must: ['ROOT', 'aux']
requests: ['ROOT', 'dobj']
.: ['ROOT', 'punct']
account: ['ROOT', 'nsubj', 'compound']
their: ['ROOT', 'dobj', 'poss']
Credit: ['ROOT', 'nsubj', 'compound', 'nmod']
and: ['ROOT', 'nsubj', 'compound', 'nmod', 'cc']
mortgage: ['ROOT', 'nsubj', 'compound', 'nmod', 'conj']


### Task A2
Extract subtree of a dependents given a token

In [ ]:
def dependents_tree(sentence: str):
  doc = doc_of(sentence)
  return {token: [*token.subtree] for token in doc}

#### Testing A2

In [ ]:
# Task A2 testing
paths = dependents_tree(example)
for token in paths:
  print(f"{token}: {paths[token]}")

Credit: [Credit, and, mortgage]
and: [and]
mortgage: [mortgage]
account: [Credit, and, mortgage, account]
holders: [Credit, and, mortgage, account, holders]
must: [must]
submit: [Credit, and, mortgage, account, holders, must, submit, their, requests, .]
their: [their]
requests: [their, requests]
.: [.]


### Task A3
Check if a given list of tokens (ordered list of words from the sentence) forms a subtree


In [6]:
def is_dependents_tree(sentence: str, sequence: [str]):
  doc = doc_of(sentence)
  sequence_set = set(sequence)
  # We use the first word as anchor to find the within-sequence root. With 
  # repetitions of the anchor word, at least one should have a within-sequence
  # root that is the root of the sequence
  anchor = sequence[0]
  for token in doc:
    if token.text == anchor:
      # Find the within-sequence root
      root = token
      while root != root.head and root.head.text in sequence_set:
        root = root.head
      # Check if the within-sequence root is the sequence root
      if sequence == [token.text for token in root.subtree]:
        return True
  return False


#### Testing A3

In [7]:
# Task A3 testing
trees = [["Credit", "and", "mortgage"],
         ["and", "Credit", "mortgage"],
         ["must", "submit", "their"]]
print(example)
for tree in trees:
  print(f"{tree}: {is_dependents_tree(example, tree)}")

Credit and mortgage account holders must submit their requests.
['Credit', 'and', 'mortgage']: True
['and', 'Credit', 'mortgage']: False
['must', 'submit', 'their']: False


### Task A4
 Identify the head of a span, given its tokens: find the head/root of a phrase

In [ ]:
def head_of(sentence: str):
  doc = doc_of(sentence)
  return doc[:].root

#### Testing A4

In [ ]:
# Task A4 testing
examples = [example, "man with a telescope", "has the tower tripped down?"]
for ex in examples:
  print(f"{ex}: {head_of(ex)}")

Credit and mortgage account holders must submit their requests.: submit
man with a telescope: man
has the tower tripped down?: tripped


### Task A5
Extract sentence subject, direct object and indirect object spans. Each span lenght is 1, for the single word.

`iobj` is [not a parsed dependency](https://spacy.io/models/en): `dative` is parsed instead.

In [18]:
def interesting_spans(sentence: str):
  doc = doc_of(sentence)
  spans = {'nsubj': [], 'dobj': [], 'dative':[]}
  for token in doc:
    if token.dep_ == 'nsubj' or token.dep_ == 'dobj' or token.dep_ == 'dative':
      subtree = [*token.subtree]
      span = doc[subtree[0].i:subtree[-1].i+1]
      spans[token.dep_].append(span)
  return spans

#### Testing A5

In [19]:
print(interesting_spans(example))
print(interesting_spans("I saw the man."))
print(interesting_spans("I read her the letter"))
print(interesting_spans("They normally give refugees shelter."))
print(interesting_spans("We booked her the cheapest morning flight to Miami."))

{'nsubj': [Credit and mortgage account holders], 'dobj': [their requests], 'dative': []}
{'nsubj': [I], 'dobj': [the man], 'dative': []}
{'nsubj': [I], 'dobj': [the letter], 'dative': [her]}
{'nsubj': [They], 'dobj': [shelter], 'dative': [refugees]}
{'nsubj': [We], 'dobj': [her, the cheapest morning flight to Miami], 'dative': []}


## Part B: Training Transition-Based Dependency Parser
This part is optional and advanced

In [ ]:
# Imports
import nltk
from nltk.parse.transitionparser import TransitionParser
from nltk.parse import DependencyEvaluator

# Download treebank
from nltk.corpus import dependency_treebank
nltk.download('dependency_treebank')

[nltk_data] Downloading package dependency_treebank to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package dependency_treebank is already up-to-date!


True

### Task B1-B3
Modify [NLTK Transition parser](https://github.com/nltk/nltk/blob/develop/nltk/parse/transitionparser.py)'s `Configuration` class to use better features.

Replace `SVM` classifier with an alternative of your choice.

In [ ]:
!pip install fasttext
!wget -nc https://github.com/MatteoZanella/NLU-assignement-1/raw/main/data/cc.en.6.bin.7z
!7za e cc.en.6.bin.7z -aos

File ‘cc.en.6.bin.7z’ already there; not retrieving.


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 100249997 bytes (96 MiB)

Extracting archive: cc.en.6.bin.7z
--
Path = cc.en.6.bin.7z
Type = 7z
Physical Size = 100249997
Headers Size = 130
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      4% . cc.en.6.bin                    9% . cc.en.6.bin                   13% . cc.en.6.bin                   18% . cc.en.6.bin                   21% . cc.en.6.bin                   23% . cc.en.6.bin                   25% . cc.en.6.bin  

In [ ]:
### Not executable on the Colab notebook - Snippet to convert the 300 features model into a 6 features model ###
# import fasttext
# import fasttext.util
# fasttext.util.download_model('en', if_exists='ignore')
# ft = fasttext.load_model('cc.en.300.bin')
# fasttext.util.reduce_model(ft, 6)
# ft.save_model('cc.en.6.bin')

In [ ]:
import tempfile
import pickle

from os import remove
from copy import deepcopy
from operator import itemgetter
import numpy as np
from numpy import array
from scipy import sparse
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder

import fasttext
import fasttext.util


from nltk.parse import ParserI, DependencyGraph, DependencyEvaluator


class NxtConfiguration(object):
    """
    Class for holding configuration which is the partial analysis of the input sentence.
    The transition based parser aims at finding set of operators that transfer the initial
    configuration to the terminal configuration.
    The configuration includes:
        - Stack: for storing partially proceeded words
        - Buffer: for storing remaining input words
        - Set of arcs: for storing partially built dependency tree
    This class also provides a method to represent a configuration as list of features.
    """

    def __init__(self, dep_graph):
        """
        :param dep_graph: the representation of an input in the form of dependency graph.
        :type dep_graph: DependencyGraph where the dependencies are not specified.
        """
        # dep_graph.nodes contain list of token for a sentence
        self.stack = [0]  # The root element
        self.buffer = list(range(1, len(dep_graph.nodes)))  # The rest is in the buffer
        self.arcs = []  # empty set of arc
        self._tokens = dep_graph.nodes
        self._max_address = len(self.buffer)

    def __str__(self):
        return (
            "Stack : "
            + str(self.stack)
            + "  Buffer : "
            + str(self.buffer)
            + "   Arcs : "
            + str(self.arcs)
        )

    def _check_informative(self, feat, flag=False):
        """
        Check whether a feature is informative
        The flag control whether "_" is informative or not
        """
        if feat is None or feat == "" or (not flag and feat == "_"):
            return None
        else:
          return feat

    def extract_features(self, stack_count=2, buffer_count=3):
        """
        Extract the set of features for the current configuration.
        :return: list(str)
        """
        # Stack to stack of tokens
        stack_tokens = [self._check_informative(self._tokens[idx]['word']) for idx in self.stack]
        # Extend with empty if too small
        for _ in range(len(stack_tokens), stack_count+1):
          stack_tokens.insert(0, None)
        # Compact the ones outside the count as a single list
        tail = [stack_tokens[:-stack_count]]
        stack_tokens = stack_tokens[-stack_count:]
        stack_tokens.extend(tail)

        # Buffer to buffer of tokens
        buffer_tokens = [self._check_informative(self._tokens[idx]['word']) for idx in self.buffer]
        # Extend with empty if too small
        for _ in range(len(buffer_tokens), buffer_count+1):
          buffer_tokens.append(None)
        # Compact the ones outside the count as a single list
        tail = [buffer_tokens[buffer_count:]]
        buffer_tokens = buffer_tokens[:buffer_count]
        buffer_tokens.extend(tail)

        # Stack POS
        stack_tags = [self._check_informative(self._tokens[idx]['tag']) for idx in self.stack[-stack_count:]]
        for _ in range(len(stack_tags), stack_count+1):
          stack_tags.insert(0, None)
        # Buffer POS
        buffer_tags = [self._check_informative(self._tokens[idx]['tag']) for idx in self.buffer[:buffer_count]]
        for _ in range(len(buffer_tags), buffer_count+1):
          buffer_tags.append(None)

        stack_tokens.extend(buffer_tokens)
        stack_tokens.extend(stack_tags)
        stack_tokens.extend(buffer_tags)
        return stack_tokens


class Transition(object):
    """
    This class defines a set of transition which is applied to a configuration to get another configuration
    Note that for different parsing algorithm, the transition is different.
    """

    # Define set of transitions
    LEFT_ARC = "LEFTARC"
    RIGHT_ARC = "RIGHTARC"
    SHIFT = "SHIFT"
    REDUCE = "REDUCE"

    def __init__(self, alg_option):
        """
        :param alg_option: the algorithm option of this parser. Currently support `arc-standard` and `arc-eager` algorithm
        :type alg_option: str
        """
        self._algo = alg_option
        if alg_option not in [
            TransitionParser.ARC_STANDARD,
            TransitionParser.ARC_EAGER,
        ]:
            raise ValueError(
                " Currently we only support %s and %s "
                % (TransitionParser.ARC_STANDARD, TransitionParser.ARC_EAGER)
            )

    def left_arc(self, conf, relation):
        """
        Note that the algorithm for left-arc is quite similar except for precondition for both arc-standard and arc-eager
            :param configuration: is the current configuration
            :return : A new configuration or -1 if the pre-condition is not satisfied
        """
        if (len(conf.buffer) <= 0) or (len(conf.stack) <= 0):
            return -1
        if conf.buffer[0] == 0:
            # here is the Root element
            return -1

        idx_wi = conf.stack[len(conf.stack) - 1]

        flag = True
        if self._algo == TransitionParser.ARC_EAGER:
            for (idx_parent, r, idx_child) in conf.arcs:
                if idx_child == idx_wi:
                    flag = False

        if flag:
            conf.stack.pop()
            idx_wj = conf.buffer[0]
            conf.arcs.append((idx_wj, relation, idx_wi))
        else:
            return -1

    def right_arc(self, conf, relation):
        """
        Note that the algorithm for right-arc is DIFFERENT for arc-standard and arc-eager
            :param configuration: is the current configuration
            :return : A new configuration or -1 if the pre-condition is not satisfied
        """
        if (len(conf.buffer) <= 0) or (len(conf.stack) <= 0):
            return -1
        if self._algo == TransitionParser.ARC_STANDARD:
            idx_wi = conf.stack.pop()
            idx_wj = conf.buffer[0]
            conf.buffer[0] = idx_wi
            conf.arcs.append((idx_wi, relation, idx_wj))
        else:  # arc-eager
            idx_wi = conf.stack[len(conf.stack) - 1]
            idx_wj = conf.buffer.pop(0)
            conf.stack.append(idx_wj)
            conf.arcs.append((idx_wi, relation, idx_wj))

    def reduce(self, conf):
        """
        Note that the algorithm for reduce is only available for arc-eager
            :param configuration: is the current configuration
            :return : A new configuration or -1 if the pre-condition is not satisfied
        """

        if self._algo != TransitionParser.ARC_EAGER:
            return -1
        if len(conf.stack) <= 0:
            return -1

        idx_wi = conf.stack[len(conf.stack) - 1]
        flag = False
        for (idx_parent, r, idx_child) in conf.arcs:
            if idx_child == idx_wi:
                flag = True
        if flag:
            conf.stack.pop()  # reduce it
        else:
            return -1

    def shift(self, conf):
        """
        Note that the algorithm for shift is the SAME for arc-standard and arc-eager
            :param configuration: is the current configuration
            :return : A new configuration or -1 if the pre-condition is not satisfied
        """
        if len(conf.buffer) <= 0:
            return -1
        idx_wi = conf.buffer.pop(0)
        conf.stack.append(idx_wi)


class NxtTransitionParser(ParserI):

    """
    Class for transition based parser. Implement "arc-eager"
    """

    ARC_EAGER = "arc-eager"

    def __init__(self, classifier='svm'):
        self._classifier = classifier
        self._algorithm = self.ARC_EAGER
        self._ft = fasttext.load_model('cc.en.6.bin')
        self._label_encoder = preprocessing.LabelEncoder()
        self._pos_encoder = OneHotEncoder(handle_unknown='ignore')
        self._pos_encoder.fit([['CC'],[','],['CD'],['DT'],['EX'],['FW'],['IN'],
                               ['JJ'],['JJR'],['JJS'],['LS'],['MD'],['NN'],
                               ['NNPS'],['NNP'],['NNPS'],['PDT'],['POS'],
                               ['PRP'],['PRP$'],['RB'],['RBR'],['RBS'],['RP'],
                               ['SYM'],['TO'],['UH'],['VB'],['VBD'],['VBG'],
                               ['VBN'],['VBP'],['VBZ'],['WP'],['WDT'],['WP$'],['WRB']])


    def _get_dep_relation(self, idx_parent, idx_child, depgraph):
        p_node = depgraph.nodes[idx_parent]
        c_node = depgraph.nodes[idx_child]

        if c_node["word"] is None:
            return None  # Root word

        if c_node["head"] == p_node["address"]:
            return c_node["rel"]
        else:
            return None

    def _convert_to_binary_features(self, features, discount=0.6):
        """
        :param features: list of feature string which is needed to convert to binary features
        :type features: list(str)
        :return : string of binary features in libsvm format  which is 'featureID:value' pairs
        """
        TAGS_COUNT = 6
        for idx, feature in enumerate(features):
          if idx >= len(features) - TAGS_COUNT:
            features[idx] = self._pos_encoder.transform([[feature]]).toarray()
          if type(feature) is list:
            discounts = np.power(discount, np.arange(0,len(feature)))[:,np.newaxis]
            feature = np.array([self._to_fasttext(token) for token in feature]) * discounts
            features[idx] = np.sum(feature, axis=0)
          else:
            features[idx] = self._to_fasttext(feature)

        return np.array(features).flatten()
    
    def _to_fasttext(self, token):
      if token is None:
        return np.zeros(self._ft.get_dimension())
      else:
        return self._ft[token]

    def _is_projective(self, depgraph):
        arc_list = []
        for key in depgraph.nodes:
            node = depgraph.nodes[key]

            if "head" in node:
                childIdx = node["address"]
                parentIdx = node["head"]
                if parentIdx is not None:
                    arc_list.append((parentIdx, childIdx))

        for (parentIdx, childIdx) in arc_list:
            # Ensure that childIdx < parentIdx
            if childIdx > parentIdx:
                temp = childIdx
                childIdx = parentIdx
                parentIdx = temp
            for k in range(childIdx + 1, parentIdx):
                for m in range(len(depgraph.nodes)):
                    if (m < childIdx) or (m > parentIdx):
                        if (k, m) in arc_list:
                            return False
                        if (m, k) in arc_list:
                            return False
        return True

    def _training_set(self, depgraphs):
        """
        Create the training set.
        """
        operation = Transition(self.ARC_EAGER)
        x_train = []
        y_train = []

        for depgraph in depgraphs:
            if not self._is_projective(depgraph):
                continue

            conf = NxtConfiguration(depgraph)
            while len(conf.buffer) > 0:
                b0 = conf.buffer[0]
                features = conf.extract_features()
                binary_features = self._convert_to_binary_features(features)
                
                x_train.append(binary_features)
                if len(conf.stack) > 0:
                    s0 = conf.stack[len(conf.stack) - 1]
                    # Left-arc operation
                    rel = self._get_dep_relation(b0, s0, depgraph)
                    if rel is not None:
                        key = Transition.LEFT_ARC + ":" + rel
                        operation.left_arc(conf, rel)
                        y_train.append(key)
                        continue

                    # Right-arc operation
                    rel = self._get_dep_relation(s0, b0, depgraph)
                    if rel is not None:
                        key = Transition.RIGHT_ARC + ":" + rel
                        operation.right_arc(conf, rel)
                        y_train.append(key)
                        continue

                    # reduce operation
                    flag = False
                    for k in range(s0):
                        if self._get_dep_relation(k, b0, depgraph) is not None:
                            flag = True
                        if self._get_dep_relation(b0, k, depgraph) is not None:
                            flag = True
                    if flag:
                        key = Transition.REDUCE
                        operation.reduce(conf)
                        y_train.append(key)
                        continue

                # Shift operation as the default
                key = Transition.SHIFT
                operation.shift(conf)
                y_train.append(key)

        x_train = np.array(x_train)
        y_train = np.array(y_train)
        self._label_encoder.fit(y_train)
        y_train = self._label_encoder.transform(y_train)
        return x_train, y_train

    def train(self, depgraphs, modelfile, verbose=True):
        """
        :param depgraphs : list of DependencyGraph as the training data
        :type depgraphs : DependencyGraph
        :param modelfile : file name to save the trained model
        :type modelfile : str
        """
        
        # Load the training set
        x_train, y_train = self._training_set(depgraphs)

        # Fit the model
        if self._classifier == 'mlp':
          model = MLPClassifier(solver='adam', alpha=1e-6, learning_rate='adaptive', learning_rate_init=0.01, hidden_layer_sizes=(64, 32, 16), random_state=1)
        elif self._classifier == 'svm':
          model = SVC(class_weight='balanced')
        else:
          raise Exception(f"No model with such name: {self._classifier}. Try 'svm' or 'mlp'")
        model.fit(x_train, y_train)
        # Save the model to file name (as pickle)
        pickle.dump(model, open(modelfile, "wb"))

    def parse(self, depgraphs, modelFile):
        """
        :param depgraphs: the list of test sentence, each sentence is represented as a dependency graph where the 'head' information is dummy
        :type depgraphs: list(DependencyGraph)
        :param modelfile: the model file
        :type modelfile: str
        :return: list (DependencyGraph) with the 'head' and 'rel' information
        """
        result = []
        # First load the model
        model = pickle.load(open(modelFile, "rb"))
        operation = Transition(self._algorithm)

        for depgraph in depgraphs:
            conf = NxtConfiguration(depgraph)
            while len(conf.buffer) > 0:
                x_test = self._convert_to_binary_features(conf.extract_features()).reshape(1, -1)
                # y_proba = np.squeeze(model.predict_proba(x_test))
                # y_classes = model.classes_[np.flip(np.argsort(y_proba))]
                # # From the prediction match to the operation
                # y_classes = self._label_encoder.inverse_transform(y_classes)
                # Note that SHIFT is always a valid operation

                y_pred = model.predict(x_test)
                y_class = np.squeeze(self._label_encoder.inverse_transform(y_pred)).item()
                
                baseTransition = y_class.split(":")[0]
                
                operationExecuted = False

                if baseTransition == Transition.LEFT_ARC:
                    if (operation.left_arc(conf, y_class.split(":")[1]) != -1):
                        operationExecuted = True
                if baseTransition == Transition.RIGHT_ARC:
                    if (operation.right_arc(conf, y_class.split(":")[1]) != -1):
                        operationExecuted = True
                if baseTransition == Transition.REDUCE:
                    if operation.reduce(conf) != -1:
                        operationExecuted = True
                if baseTransition == Transition.SHIFT or not operationExecuted:
                    operation.shift(conf)

                # for y_pred in y_classes:

                #     baseTransition = y_pred.split(":")[0]

                #     if baseTransition == Transition.LEFT_ARC:
                #         if (operation.left_arc(conf, y_pred.split(":")[1]) != -1):
                #             break
                #     elif baseTransition == Transition.RIGHT_ARC:
                #         if (operation.right_arc(conf, y_pred.split(":")[1]) != -1):
                #             break
                #     elif baseTransition == Transition.REDUCE:
                #         if operation.reduce(conf) != -1:
                #             break
                #     elif baseTransition == Transition.SHIFT:
                #         if operation.shift(conf) != -1:
                #             break
            # Finish with operations build the dependency graph from Conf.arcs

            new_depgraph = deepcopy(depgraph)
            for key in new_depgraph.nodes:
                node = new_depgraph.nodes[key]
                node["rel"] = ""
                # With the default, all the token depend on the Root
                node["head"] = 0
            for (head, rel, child) in conf.arcs:
                c_node = new_depgraph.nodes[child]
                c_node["head"] = head
                c_node["rel"] = rel
            result.append(new_depgraph)

        return result



### Task B2
Evaluate the features, comparing the performance to the original.

In [ ]:
VALIDATION_COUNT = 200

In [ ]:
# Standard Transition Parser
std_tp = TransitionParser('arc-eager')

TRAINING_COUNT = 100

std_tp.train(dependency_treebank.parsed_sents()[:TRAINING_COUNT], 'tp.std-model')

# Performances
parses = std_tp.parse(dependency_treebank.parsed_sents()[-VALIDATION_COUNT:], 'tp.std-model')
de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-VALIDATION_COUNT:])
std_uas = de.eval()[1]
print(f"Original parser: {std_uas}")

 Number of training examples : 100
 Number of valid (projective) examples : 100
[LibSVM]Original parser: 0.750459981600736


In [ ]:
# Advanced Transition Parser with SVM Classifier
nxt_tp = NxtTransitionParser('svm')

TRAINING_COUNT = 100

nxt_tp.train(dependency_treebank.parsed_sents()[:TRAINING_COUNT], 'tp.nxt-model')

parses = nxt_tp.parse(dependency_treebank.parsed_sents()[-VALIDATION_COUNT:], 'tp.nxt-model')
de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-VALIDATION_COUNT:])
std_uas = de.eval()[1]
print(f"Advanced (SVM) parser: {std_uas}")

Advanced (SVM) parser: 0.5278288868445262


In [ ]:
# Advanced Transition Parser with MLP Classifier
nxt_tp = NxtTransitionParser('mlp')

TRAINING_COUNT = 100

nxt_tp.train(dependency_treebank.parsed_sents()[:TRAINING_COUNT], 'tp.std-model')

parses = nxt_tp.parse(dependency_treebank.parsed_sents()[-VALIDATION_COUNT:], 'tp.std-model')
de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-VALIDATION_COUNT:])
std_uas = de.eval()[1]
print(f"Advanced (MLP) parser: {std_uas}")

Advanced (MLP) parser: 0.5363385464581417


In [ ]:
# Advanced Transition Parser with MLP Classifier
nxt_tp = NxtTransitionParser('mlp')

TRAINING_COUNT = len(dependency_treebank.parsed_sents()) - VALIDATION_COUNT

nxt_tp.train(dependency_treebank.parsed_sents()[:TRAINING_COUNT], 'tp.std-model')

parses = nxt_tp.parse(dependency_treebank.parsed_sents()[-VALIDATION_COUNT:], 'tp.std-model')
de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-VALIDATION_COUNT:])
std_uas = de.eval()[1]
print(f"Advanced (MLP) parser: {std_uas}")

Advanced (MLP) parser: 0.7902483900643974
